from google.colab import drive
drive.mount('/content/drive')

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
#%tensorflow_version 2.x
#import tensorflow as tf
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

#https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=sXnDmXR7RDr2

# **MyAZNet**  


### **Libraries.** 

In [3]:
import os
import pathlib
import torch
import tarfile
import torchvision
import glob
import torch.nn as nn
from torch.optim import Adam
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.utils import download_url

In [4]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [5]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
print(device)

cuda


### **Parameters.** 

In [7]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [8]:
#Dataloader

#Path for training and testing directory
train_path='../MIT_split/train'
test_path='../MIT_split/test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=128, shuffle=True
    )

test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=128, shuffle=True
    )

In [9]:
#Image categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [18]:
print(classes)

['Opencountry', 'coast', 'forest', 'highway', 'inside_city', 'mountain', 'street', 'tallbuilding']


In [19]:
#CNN Network
class MLP(nn.Module):
    def __init__(self,num_classes=8):
        super(MLP,self).__init__()

        self.num_classes = num_classes

        #Conv1
        # self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3)
        self.conv1 = nn.Conv2d(224, 224, kernel_size=3)
        #Shape= (256,12,224,224)
        self.relu1=nn.ReLU()

        #Batch Norm
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,224,224)
        self.relu2=nn.ReLU()
        #Shape= (256,12,224,224)

        #MaxPool
        self.pool1=nn.MaxPool2d(kernel_size=2)
        #Shape= (256,12,112,112)
        
        #Conv2
        self.conv2=nn.Conv2d(in_channels=12,out_channels=32,kernel_size=3)
        #Shape= (256,32,112,112)
        self.relu3=nn.ReLU()
        #Shape= (256,32,112,112)

        #Global Avg Pooling

        #Dense
        #self.fc=nn.Linear(in_features=112 * 112 * 20, out_features=num_classes)
        #self.relu4=nn.ReLU()

        #Dropout
        #self.drop1 = nn.Dropout(p=0.4)

        #Dense
        self.fc2=nn.Linear(in_features=112 * 112 * 32,out_features=num_classes)
        self.softmax = nn.Softmax(dim=1)

       
    #Feed forwad function
        
    def forward(self,input):

        output=self.conv1(input)
        output=self.relu1(output)

        output=self.bn1(output)
        output=self.relu2(output)

        output=self.pool1(output)

        output=self.conv2(output)
        output=self.relu3(output)

        #output=self.fc(output)
        #output=self.relu4(output)

        #output=self.drop1(output)

        output=output.view(output.size(0),-1)

        output=self.fc2(output)            

        output=self.softmax(output)
            
        return output

In [20]:
model=MLP(num_classes=8).to(device)

TypeError: __init__() got multiple values for argument 'in_channels'

In [13]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001)
loss_function=nn.CrossEntropyLoss()

In [14]:
num_epochs=2

In [15]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [16]:
print(train_count,test_count)

1881 807


In [17]:
#Model training and saving best model
#source: https://github.com/gaurav67890/Pytorch_Tutorials/blob/master/cnn-scratch-training.ipynb

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    print('testing')

    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0

    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

RuntimeError: mat1 dim 1 must match mat2 dim 0

In [ ]:
#

# **Hyperparametrization**

In [ ]:
# list all data in history
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig('../logs/plots_full_data/accuracy_hyper.jpg')
#plt.close()

In [ ]:
  # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('../logs/plots_full_data/loss_hyper.jpg')